# 31-feature-engineering-round3

Notebook description: This notebook creates functions that split our time frames into different files and saves the files using the feature engineering that is generated in notebook 30. This notebook is for local paths, not ACCRE. Otherwise, is the same as round2 notebook.

In [ ]:
# import packages
import pandas as pd
#import janitor
import numpy as np
import os
# must install janitor package with the following shell command:
# 'conda install -c conda-forge pyjanitor'

# import label encoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
if not os.path.exists("bridgestone-mubarak/fe_data"):
    os.mkdir("bridgestone-mubarak/fe_data")

FileNotFoundError: [Errno 2] No such file or directory: 'bridgestone-mubarak/fe_data'

In [ ]:
df = mega_table

In [ ]:
df.head()

,unnamed_0_x,store_id,tran_id,date,article_id,indiv_id,vehicle_id,units,sales,prod_group_code,prod_group_desc,category_code,category_desc,segment_code,segment_desc,class_code,class_desc,discount_flag,cross_section,aspect_ratio,rim_size,state_code,zip_code,msa_,unnamed_0_y,email_optin_ind,ah1_res_bus_indc,supp1_bus_pander,unnamed_0,make,model,sub_model,model_year
0,1,337934,990994590,2018-07-23,7001647,318422234.0,944814728,0.0,-8.55,4.0,Services,78.0,Heating/Cooling Systems,22008.0,Discount Heating/Cooling,22010.0,Discount Heating/Cooling,Y,NaN,NaN,NaN,MD,21740,"HAGERSTOWN,",11239886,Y,R,N,NaN,NaN,NaN,NaN,NaN
1,2,337934,990994590,2018-07-23,7001715,318422234.0,944814728,0.0,-50.45,4.0,Services,78.0,Heating/Cooling Systems,105.0,Air Conditioning Services,371.0,A/C Inspection,Y,NaN,NaN,NaN,MD,21740,"HAGERSTOWN,",11239886,Y,R,N,NaN,NaN,NaN,NaN,NaN
2,3,337934,990994590,2018-07-23,7096547,318422234.0,944814728,0.0,10.00,4.0,Services,78.0,Heating/Cooling Systems,105.0,Air Conditioning Services,386.0,A/C System Leak Detection,N,NaN,NaN,NaN,MD,21740,"HAGERSTOWN,",11239886,Y,R,N,NaN,NaN,NaN,NaN,NaN
3,4,337934,990994590,2018-07-23,7001647,318422234.0,944814728,0.0,0.00,4.0,Services,78.0,Heating/Cooling Systems,22008.0,Discount Heating/Cooling,22010.0,Discount Heating/Cooling,Y,NaN,NaN,NaN,MD,21740,"HAGERSTOWN,",11239886,Y,R,N,NaN,NaN,NaN,NaN,NaN
4,5,337934,990994590,2018-07-23,7001715,318422234.0,944814728,0.0,0.00,4.0,Services,78.0,Heating/Cooling Systems,105.0,Air Conditioning Services,371.0,A/C Inspection,Y,NaN,NaN,NaN,MD,21740,"HAGERSTOWN,",11239886,Y,R,N,NaN,NaN,NaN,NaN,NaN


In [ ]:
df["month_year"] = df["date"].str[:-3]

In [ ]:
date_list = sorted(list(df["month_year"].unique()))

In [ ]:
date_list

['2015-04',
 '2015-05',
 '2015-06',
 '2015-07',
 '2015-08',
 '2015-09',
 '2015-10',
 '2015-11',
 '2015-12',
 '2016-01',
 '2016-02',
 '2016-03',
 '2016-04',
 '2016-05',
 '2016-06',
 '2016-07',
 '2016-08',
 '2016-09',
 '2016-10',
 '2016-11',
 '2016-12',
 '2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10']

#### 2015 to 2017

In [ ]:
d2015_17 = date_list[:date_list.index('2018-01')]

#### 2015 to 2018-02

In [ ]:
d2015_1802 = date_list[:date_list.index('2018-03')]

#### 2015 to 2018-04

In [ ]:
d2015_1804 = date_list[:date_list.index('2018-05')]

#### 2015 to 2018-06

In [ ]:
d2015_1806 = date_list[:date_list.index('2018-07')]

#### 2015 to 2018-08

In [ ]:
d2015_1808 = date_list[:date_list.index('2018-09')]

#### 2015 to 2018-10

In [ ]:
d2015_1810 = date_list

#### Function for collecting data

In [ ]:
## This should be used for everything except the 2015 to October 2018 data
def data_collection(previous_months_list, response_month):
    df1 = df[df['date'].str[:-3].isin(previous_months_list)].reset_index()
    useless_columns = ["unnamed_0", "unnamed_0_x", "unnamed_0_y"]
    for useless in useless_columns:
        if useless in list(df1.columns):
            df1 = df1.drop(useless, axis = 1)
    dfc = df1.groupby(['indiv_id'
                              ,'tran_id'
                              ,'date'
                              ,'prod_group_desc'
                              ,'vehicle_id'], as_index = False).agg(
        rows_collapsed = pd.NamedAgg(column='article_id', aggfunc='count')
        ,sales_total = pd.NamedAgg(column='sales', aggfunc='sum')
        ,model_year_avg = pd.NamedAgg(column='model_year', aggfunc='mean')
        ,region = pd.NamedAgg(column='msa_', aggfunc='max'))
    # create three additional columns in the dataframe 
    # each has a 1 or a 0 depending on what kind of product transaction it was
    dfc['tires'] = np.where(dfc['prod_group_desc']== 'Tires', 1, 0)
    dfc['services'] = np.where(dfc['prod_group_desc']== 'Services', 1, 0)
    dfc['other'] = np.where(dfc['prod_group_desc']== 'Other', 1, 0)
    
    # add monthly variables
    dfc['jan'] = np.where((dfc['date'].str[5:7].isin(['01']) & dfc['tires'] == 1), 1, 0)
    dfc['feb'] = np.where((dfc['date'].str[5:7].isin(['02']) & dfc['tires'] == 1), 1, 0)
    dfc['mar'] = np.where((dfc['date'].str[5:7].isin(['03']) & dfc['tires'] == 1), 1, 0)
    dfc['apr'] = np.where((dfc['date'].str[5:7].isin(['04']) & dfc['tires'] == 1), 1, 0)
    dfc['may'] = np.where((dfc['date'].str[5:7].isin(['05']) & dfc['tires'] == 1), 1, 0)
    dfc['jun'] = np.where((dfc['date'].str[5:7].isin(['06']) & dfc['tires'] == 1), 1, 0)
    dfc['jul'] = np.where((dfc['date'].str[5:7].isin(['07']) & dfc['tires'] == 1), 1, 0)
    dfc['aug'] = np.where((dfc['date'].str[5:7].isin(['08']) & dfc['tires'] == 1), 1, 0)
    dfc['sep'] = np.where((dfc['date'].str[5:7].isin(['09']) & dfc['tires'] == 1), 1, 0)
    dfc['oct'] = np.where((dfc['date'].str[5:7].isin(['10']) & dfc['tires'] == 1), 1, 0)
    dfc['nov'] = np.where((dfc['date'].str[5:7].isin(['11']) & dfc['tires'] == 1), 1, 0)
    dfc['dec'] = np.where((dfc['date'].str[5:7].isin(['12']) & dfc['tires'] == 1), 1, 0)
    
    # recast date as datetime
    dfc['date'] = pd.to_datetime(dfc['date'], format = '%Y-%m-%d')

    # add end date for each row (same)
    response_date = response_month + "-01"
    dfc['end_date'] = pd.to_datetime(response_date, format = '%Y-%m-%d')
    
    # add 'days_since' measure
    # this is days since the transaction took place, from the perspective of 2018-01-01
    dfc['days_since_any'] = (dfc['end_date'] - dfc['date']).dt.days
    dfc['days_since_tires'] = np.where(dfc['tires']== 1, dfc['days_since_any'], None)
    dfc['days_since_services'] = np.where(dfc['services']== 1, dfc['days_since_any'], None)
    dfc['days_since_other'] = np.where(dfc['other']== 1, dfc['days_since_any'], None)
    
    # label encode region
    le = LabelEncoder()
    dfc.region = le.fit_transform(dfc.region)
    
    dfc2 = dfc.groupby(['indiv_id'], as_index = False).agg(
        total_transaction = pd.NamedAgg(column='tran_id', aggfunc='count')
        ,sales_total = pd.NamedAgg(column='sales_total', aggfunc='sum')
        ,tire_purchases = pd.NamedAgg(column='tires', aggfunc='sum')
        ,service_purchases = pd.NamedAgg(column='services', aggfunc='sum')
        ,other_purchases = pd.NamedAgg(column='other', aggfunc='sum')
        ,days_since_first_transaction = pd.NamedAgg(column='days_since_any', aggfunc='max')
        ,days_since_last_transaction = pd.NamedAgg(column='days_since_any', aggfunc='min')
        ,days_since_first_tire_purchase = pd.NamedAgg(column='days_since_tires', aggfunc='max')
        ,days_since_last_tire_purchase = pd.NamedAgg(column='days_since_tires', aggfunc='min')
        ,vehicle_count = pd.NamedAgg(column='vehicle_id', aggfunc='nunique')
        ,model_year_avg = pd.NamedAgg(column='model_year_avg', aggfunc='mean')
        ,region = pd.NamedAgg(column='region', aggfunc='max')
        ,jan = pd.NamedAgg(column='jan', aggfunc='sum')
        ,feb = pd.NamedAgg(column='feb', aggfunc='sum')
        ,mar = pd.NamedAgg(column='mar', aggfunc='sum')
        ,apr = pd.NamedAgg(column='apr', aggfunc='sum')
        ,may = pd.NamedAgg(column='may', aggfunc='sum')
        ,jun = pd.NamedAgg(column='jun', aggfunc='sum')
        ,jul = pd.NamedAgg(column='jul', aggfunc='sum')
        ,aug = pd.NamedAgg(column='aug', aggfunc='sum')
        ,sep = pd.NamedAgg(column='sep', aggfunc='sum')
        ,oct = pd.NamedAgg(column='oct', aggfunc='sum')
        ,nov = pd.NamedAgg(column='nov', aggfunc='sum')
        ,dec = pd.NamedAgg(column='dec', aggfunc='sum'))
    # tire purchasing frequency
    dfc2['tire_purchase_freq'] = dfc2['tire_purchases'] / dfc2['days_since_first_transaction']
    dfc2['days_since_first_tire_purchase'] = dfc2['days_since_first_tire_purchase'].fillna(-1)
    dfc2['days_since_last_tire_purchase'] = dfc2['days_since_last_tire_purchase'].fillna(-1)    
    
    # subsetting to 2018-01 and filtering for tire purchases
    df_response = df[df['date'].str[:7].isin([response_month])].reset_index()
    df_response = df_response[(df_response['prod_group_desc'] == 'Tires')]
    purchaser_ids = df_response['indiv_id'].unique().tolist()
    
    # adding response column to feature dataset
    dfc2['response'] = np.where(dfc2['indiv_id'].isin(purchaser_ids), 1, 0)
    
    return dfc2

In [ ]:
## This should be used for the 2015 to October 2018 data set
def data_collection_special(previous_months_list):
    df1 = df[df['date'].str[:-3].isin(previous_months_list)].reset_index()
    useless_columns = ["unnamed_0", "unnamed_0_x", "unnamed_0_y"]
    for useless in useless_columns:
        if useless in list(df1.columns):
            df1 = df1.drop(useless, axis = 1)
    dfc = df1.groupby(['indiv_id'
                              ,'tran_id'
                              ,'date'
                              ,'prod_group_desc'
                              ,'vehicle_id'], as_index = False).agg(
        rows_collapsed = pd.NamedAgg(column='article_id', aggfunc='count')
        ,sales_total = pd.NamedAgg(column='sales', aggfunc='sum')
        ,model_year_avg = pd.NamedAgg(column='model_year', aggfunc='mean')
        ,region = pd.NamedAgg(column='msa_', aggfunc='max'))
    # create three additional columns in the dataframe 
    # each has a 1 or a 0 depending on what kind of product transaction it was
    dfc['tires'] = np.where(dfc['prod_group_desc']== 'Tires', 1, 0)
    dfc['services'] = np.where(dfc['prod_group_desc']== 'Services', 1, 0)
    dfc['other'] = np.where(dfc['prod_group_desc']== 'Other', 1, 0)
    
    # add monthly variables
    dfc['jan'] = np.where((dfc['date'].str[5:7].isin(['01']) & dfc['tires'] == 1), 1, 0)
    dfc['feb'] = np.where((dfc['date'].str[5:7].isin(['02']) & dfc['tires'] == 1), 1, 0)
    dfc['mar'] = np.where((dfc['date'].str[5:7].isin(['03']) & dfc['tires'] == 1), 1, 0)
    dfc['apr'] = np.where((dfc['date'].str[5:7].isin(['04']) & dfc['tires'] == 1), 1, 0)
    dfc['may'] = np.where((dfc['date'].str[5:7].isin(['05']) & dfc['tires'] == 1), 1, 0)
    dfc['jun'] = np.where((dfc['date'].str[5:7].isin(['06']) & dfc['tires'] == 1), 1, 0)
    dfc['jul'] = np.where((dfc['date'].str[5:7].isin(['07']) & dfc['tires'] == 1), 1, 0)
    dfc['aug'] = np.where((dfc['date'].str[5:7].isin(['08']) & dfc['tires'] == 1), 1, 0)
    dfc['sep'] = np.where((dfc['date'].str[5:7].isin(['09']) & dfc['tires'] == 1), 1, 0)
    dfc['oct'] = np.where((dfc['date'].str[5:7].isin(['10']) & dfc['tires'] == 1), 1, 0)
    dfc['nov'] = np.where((dfc['date'].str[5:7].isin(['11']) & dfc['tires'] == 1), 1, 0)
    dfc['dec'] = np.where((dfc['date'].str[5:7].isin(['12']) & dfc['tires'] == 1), 1, 0)
    
    # recast date as datetime
    dfc['date'] = pd.to_datetime(dfc['date'], format = '%Y-%m-%d')

    # add end date for each row (same)
    dfc['end_date'] = pd.to_datetime('2018-11-01', format = '%Y-%m-%d')
    
    # add 'days_since' measure
    # this is days since the transaction took place, from the perspective of 2018-01-01
    dfc['days_since_any'] = (dfc['end_date'] - dfc['date']).dt.days
    dfc['days_since_tires'] = np.where(dfc['tires']== 1, dfc['days_since_any'], None)
    dfc['days_since_services'] = np.where(dfc['services']== 1, dfc['days_since_any'], None)
    dfc['days_since_other'] = np.where(dfc['other']== 1, dfc['days_since_any'], None)
    
    # label encode region
    le = LabelEncoder()
    dfc.region = le.fit_transform(dfc.region)
    
    dfc2 = dfc.groupby(['indiv_id'], as_index = False).agg(
        total_transaction = pd.NamedAgg(column='tran_id', aggfunc='count')
        ,sales_total = pd.NamedAgg(column='sales_total', aggfunc='sum')
        ,tire_purchases = pd.NamedAgg(column='tires', aggfunc='sum')
        ,service_purchases = pd.NamedAgg(column='services', aggfunc='sum')
        ,other_purchases = pd.NamedAgg(column='other', aggfunc='sum')
        ,days_since_first_transaction = pd.NamedAgg(column='days_since_any', aggfunc='max')
        ,days_since_last_transaction = pd.NamedAgg(column='days_since_any', aggfunc='min')
        ,days_since_first_tire_purchase = pd.NamedAgg(column='days_since_tires', aggfunc='max')
        ,days_since_last_tire_purchase = pd.NamedAgg(column='days_since_tires', aggfunc='min')
        ,vehicle_count = pd.NamedAgg(column='vehicle_id', aggfunc='nunique')
        ,model_year_avg = pd.NamedAgg(column='model_year_avg', aggfunc='mean')
       ,region = pd.NamedAgg(column='region', aggfunc='max')
        ,jan = pd.NamedAgg(column='jan', aggfunc='sum')
        ,feb = pd.NamedAgg(column='feb', aggfunc='sum')
        ,mar = pd.NamedAgg(column='mar', aggfunc='sum')
        ,apr = pd.NamedAgg(column='apr', aggfunc='sum')
        ,may = pd.NamedAgg(column='may', aggfunc='sum')
        ,jun = pd.NamedAgg(column='jun', aggfunc='sum')
        ,jul = pd.NamedAgg(column='jul', aggfunc='sum')
        ,aug = pd.NamedAgg(column='aug', aggfunc='sum')
        ,sep = pd.NamedAgg(column='sep', aggfunc='sum')
        ,oct = pd.NamedAgg(column='oct', aggfunc='sum')
        ,nov = pd.NamedAgg(column='nov', aggfunc='sum')
        ,dec = pd.NamedAgg(column='dec', aggfunc='sum'))

    # tire purchasing frequency
    dfc2['tire_purchase_freq'] = dfc2['tire_purchases'] / dfc2['days_since_first_transaction']
    dfc2['days_since_first_tire_purchase'] = dfc2['days_since_first_tire_purchase'].fillna(-1)
    dfc2['days_since_last_tire_purchase'] = dfc2['days_since_last_tire_purchase'].fillna(-1)    
    
    return dfc2

#### Collecting data for 2015 - 2017

In [ ]:
d_2015_17 = data_collection(d2015_17, '2018-01')
d_2015_17.to_csv("fe_data/2015_17.csv", index = False)

#### Collecting data for 2015 - 2018-02

In [ ]:
df2015_18_02 = data_collection(d2015_1802, '2018-03')
df2015_18_02.to_csv("fe_data/2015_18_02.csv", index = False)

#### Collecting data for 2015 - 2018-04

In [ ]:
df2015_18_04 = data_collection(d2015_1804, '2018-05')
df2015_18_04.to_csv("fe_data/2015_18_04.csv", index = False)

#### Collecting data for 2015 - 2018-06

In [ ]:
df2015_18_06 = data_collection(d2015_1806, '2018-07')
df2015_18_06.to_csv("fe_data/2015_18_06.csv", index = False)

#### Collecting data for 2015 - 2018-08

In [ ]:
df2015_18_08 = data_collection(d2015_1808, '2018-09')
df2015_18_08.to_csv("fe_data/2015_18_08.csv", index = False)

#### Collecting data for 2015 - 2018-10

In [ ]:
df2015_18_10 = data_collection_special(d2015_1810)
df2015_18_10.to_csv("fe_data/2015_18_10.csv", index = False)